<a href="https://colab.research.google.com/github/fernandarrios/tcc_covid_recuperacao/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#To Do:
 
 - Gráfico de análise temporal da evolução dos casos;
 - Rodar código que acrescenta uma coluna de id para ordenar o gráfico; 

# Importando os dados

Dados até o dia 17/09/22

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling.profile_report import ProfileReport


def plot_chart(desired_variable, df, column, title):
    plt.figure(figsize=(12, 8))
    sns.countplot(y=desired_variable,
                  data=df,
                  palette='rocket',
                  order=column.value_counts().index).set_title(title)
    plt.show()


def fixing_data_type(column, first_symbol, second_symbol):
    try:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, format='%Y%m%d')
    except ValueError:
        column = column.apply(lambda x : str(x).replace(first_symbol, second_symbol))
        column = pd.to_datetime(column, dayfirst=True)
    return column

In [ ]:
covid_sistema_um = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_1.txt', sep='\t')
covid_sistema_dois = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_2.txt', sep='\t')
covid_sistema_tres = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/XLSX_Sistemas_3.txt', sep='\t')
covid_sistema_quatro = pd.read_csv(
    'C:/Users/nanda/PycharmProjects/puc_tcc/xlsx_sistemas.txt', sep='\t')

covid_sistema_um['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_um['DATA_NOTIFICACAO'], '-', '')
covid_sistema_dois['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_dois['DATA_NOTIFICACAO'], '-', '')
covid_sistema_tres['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_tres['DATA_NOTIFICACAO'], '-', '')
covid_sistema_quatro['DATA_NOTIFICACAO'] = fixing_data_type(
    covid_sistema_quatro['DATA_NOTIFICACAO'], '/', '-')

covid_df = pd.concat([covid_sistema_um, covid_sistema_dois, covid_sistema_tres,
                      covid_sistema_quatro])

report = ProfileReport(covid_df, title='Primeira investigação do dataset')
report.to_file('TCC_COVID_REPORT_1.html')

# Análise Exploratória

#### Eliminando colunas que não fazem sentido para o nosso problema e nem faz sentido para conhecermos o dataset

In [ ]:
covid_df.drop(columns=['ID', 'CODIGO', 'DATA_EVOLUCAO', 'MICRO', 'MACRO', 
                       'ETNIA', 'MUNICIPIO_RESIDENCIA', 'DATA_1_SINTOMA', 'URS', 
                       'DATA_ATUALIZACAO', 'CLASSIFICACAO_CASO'], inplace=True)

#### Como vimos no profile report, temos valores para idade que não fazem sentido, vamos investigar melhor

 Alguém de quase 400 anos não faz sentido, pode ter ocorrido algum erro de digitação. 
 
 Vamos analisar melhor os outliers

In [ ]:
covid_df['IDADE'].min()
covid_df['IDADE'].max()

In [ ]:
sns.boxplot(covid_df['IDADE'])
plt.show()

Tem 1127 outliers e 1825 valores nulos, no primeiro momento pensei em só pegar os dois primeiros números e considerar o terceiro como erro de digitação e, mas o preenchimento da faixa etária deve ter ocorrido de forma automática de acordo com o input manual da idade. Decidi então estabelecer um valor máximo de idade de 100 anos, e filtrar todos os valores que fogem disso.

In [ ]:
covid_df = covid_df.loc[covid_df['IDADE'] <= 100]

In [ ]:
sns.boxplot(covid_df['IDADE'])
plt.show()

#### Contagem por faixa etária 

In [ ]:
covid_df['FAIXA_ETARIA'].value_counts()
plot_chart('FAIXA_ETARIA', covid_df, covid_df['FAIXA_ETARIA'],
           'Faixa etária dos casos de covid')

Como podemos ver, as pessoas mais afetadas foram as pessoas de 30 a 39 anos

#### Qual o local que mais publicou informação sobre a covid em MG?

In [ ]:
covid_df['ORIGEM_DA_INFORMACAO'].value_counts()
plot_chart('ORIGEM_DA_INFORMACAO', covid_df, covid_df['ORIGEM_DA_INFORMACAO'],
           'Portal de Publicação')

O maior portal responsável por divulgar os números da COVID-19 em MG foi o ESUS.

#### Quantas pessoas apresentaram comorbidade?

In [ ]:
covid_df['COMORBIDADE'].value_counts()
plot_chart('COMORBIDADE', covid_df, covid_df['COMORBIDADE'],
           'Presença ou não de comorbidade')

#### Qual a raça predominante?

In [ ]:
covid_df['RACA'].value_counts()
plot_chart('RACA', covid_df, covid_df['RACA'], 'Raça dos cidadãos')

#### Quantas pessoas foram internadas?

In [ ]:
covid_df['INTERNACAO'].value_counts()
plot_chart('INTERNACAO', covid_df, covid_df['INTERNACAO'], 'Casos de internação')


#### Elas foram para o uti?

In [ ]:
covid_df['UTI'].value_counts()
plot_chart('UTI', covid_df, covid_df['UTI'], 'Contagem de casos de internação na UTI')

#### Quantas sobreviveram? Quantas morreram?

In [ ]:
covid_df['EVOLUCAO'].value_counts()
plot_chart('EVOLUCAO', covid_df, covid_df['EVOLUCAO'], 'Contagem de evolução dos casos')


Como podemos ver, temos muito mais casos de pessoas recuperadas, então teremos que fazer o balanceamento delas. O método escolhido irá ser o oversampling de óbitos.

# Preparação para o modelo

#### Eliminando colunas que não fazem sentido para o ML

In [ ]:
covid_df.drop(columns=['DATA_NOTIFICACAO', 'ORIGEM_DA_INFORMACAO'], inplace=True)